In [1]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import norm
from joblib import Parallel, delayed
from scipy.stats import qmc

num_obvs = 100_000
data = pd.read_csv("data.csv")
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
data["sec_loading"], data["t"] = r[data["sector"].values], st.norm.ppf(data.p)
factors, sample = np.random.normal(0,1, (num_obvs, len(r)+len(data))), []

def process_obs(obs):
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]
    ind = (
            r[0]**.5 * m_factor + (data.sec_loading-r[0])**.5 * sec_factor + (1-data.sec_loading)**.5 * res_factor
            < data.t
    )
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind))
    return sum(loss), np.var(loss)

answers = Parallel(n_jobs=-1)(delayed(process_obs)(obs) for obs in factors)
answers = np.array(answers)

# Wyniki
sample_losses = answers[:, 0]
sample_vars = answers[:, 1]

# Obliczenie VaR
VaR_qmc = sorted([-s for s in sample_losses])[int(0.001*num_obvs)]  # Wyciąganie 100-tego wyniku z posortowanych strat


In [2]:
VaR_qmc

-17206.931303687852

In [3]:
np.mean(sample_vars)

1360.181267462014